In [1]:
from spacy.lang.en import English
nlp = English()

C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
nlp.pipe_names

[]

In [3]:
# prepare an empty model to train
# nlp = spacy.load('en_web_core_sm')
nlp.vocab.vectors.name = 'demo'
ner = nlp.create_pipe('ner')
nlp.add_pipe('ner', last=True)

In [4]:
import os
import json

In [5]:
def load_data():
    training_data = {}
    for item in os.listdir("datasets"):
        if ".json" in item:
            temp = json.load(open(f"./datasets/{item}"))
            if training_data == {}:
                training_data = temp
            else:
                training_data["annotations"].extend(temp["annotations"])
    
    return training_data

In [16]:
json_data = load_data()

In [18]:
training_data = []
for item in json_data["annotations"]:
    training_data.append({
        "text" : item[0],
        "entities" : item[1]["entities"]
    })

In [7]:
ner = nlp.get_pipe("ner")

In [20]:
# Add the custome NER Tags as entities into the model
for label in json_data["classes"]:
    ner.add_label(label)

In [9]:
nlp.get_pipe("ner").labels

('BRAND', 'COMPONENT', 'CONFIG', 'PRICE', 'PRICE_TYPE', 'TYPE')

In [41]:
from spacy.util import minibatch, compounding
from pathlib import Path
import random

In [10]:
from helper import Verbose
verbose = Verbose()

In [11]:
epochs = 100

In [23]:
# Train the model
optimizer = nlp.begin_training()

for i in range(epochs):
    verbose.make_verbose(i+1, epochs, "training")
    for item in training_data:
        if len(item["text"]) > 0: # in case an empty sentence was saved while annotating
            nlp.update([item], sgd=optimizer)

training  [>-------------------------]  1/100 ETA: 3582 sec for 36.19 sec/steps

TypeError: [E978] The Language.update method takes a list of Example objects, but got: {<class 'dict'>}

In [22]:
item

{'text': 'What do you reccomend?', 'entities': []}

In [25]:
from spacy.tokens import Doc
from spacy.training import Example
from nltk import word_tokenize